In [ ]:
# 第二版，paada改正

import pandas as pd
import numpy as np
from fractions import Fraction
from decimal import Decimal, ROUND_HALF_UP

def generate_kp_table_paada_strict_final():
    # ==============================
    # 1. 基础常量
    # ==============================
    ZODIAC_SEC = Fraction(1296000, 1)
    SIGN_SEC   = Fraction(108000, 1)
    PAADA_SEC  = Fraction(12000, 1)   # [已修正] 3度20分 = 12000秒
    STAR_SEC   = Fraction(48000, 1)

    SIGNS = ['Ari', 'Tau', 'Gem', 'Cnc', 'Leo', 'Vir', 'Lib', 'Sco', 'Sgr', 'Cap', 'Aqr', 'Pis']

    STARS = [
        'Ashwini', 'Bharani', 'Krittika', 'Rohini', 'Mrigashira', 'Ardra',
        'Punarvasu', 'Pushya', 'Ashlesha', 'Magha', 'Purva Phalguni', 'Uttara Phalguni',
        'Hasta', 'Chitra', 'Swati', 'Vishakha', 'Anuradha', 'Jyeshtha',
        'Mula', 'Purva Ashadha', 'Uttara Ashadha', 'Shravana', 'Dhanishta', 'Shatabhisha',
        'Purva Bhadrapada', 'Uttara Bhadrapada', 'Revati'
    ]

    LORDS = ['Ke', 'Ve', 'Su', 'Mo', 'Ma', 'Ra', 'Ju', 'Sa', 'Me']
    YEARS = {'Ke': 7, 'Ve': 20, 'Su': 6, 'Mo': 10, 'Ma': 7, 'Ra': 18, 'Ju': 16, 'Sa': 19, 'Me': 17}

    SIGN_LORDS_MAP = {
    'Ari': 'Ma', 'Tau': 'Ve', 'Gem': 'Me', 'Cnc': 'Mo', 'Leo': 'Su', 'Vir': 'Me',
    'Lib': 'Ve', 'Sco': 'Ma', 'Sgr': 'Ju', 'Cap': 'Sa', 'Aqr': 'Sa', 'Pis': 'Ju'
    }

    # ==============================
    # 2. 核心辅助函数
    # ==============================
    def get_lord_sequence(start_lord):
        idx = LORDS.index(start_lord)
        return LORDS[idx:] + LORDS[:idx]

    def strict_round(val_fraction):
        return int(Decimal(str(float(val_fraction))).quantize(0, ROUND_HALF_UP))

    def format_row(abs_start, abs_end, sign_idx, star_name, star_lord, sub_lord, ss_lord, cil_n, ks_n, ks_d, paada_val):
        s_sec = strict_round(abs_start)
        e_sec = strict_round(abs_end)

        deg_from = s_sec / 3600.0
        deg_to = e_sec / 3600.0

        # Sign 内的度分秒计算
        sign_start_sec = sign_idx * 108000
        rel_sec = s_sec - sign_start_sec

        d = rel_sec // 3600
        m = (rel_sec % 3600) // 60
        s = rel_sec % 60

        sign_name = SIGNS[sign_idx]

        return {
            'From': deg_from,
            'To': deg_to,
            'Sign': sign_name,
            'Star': star_name,
            'Sign-Lord': SIGN_LORDS_MAP[sign_name],
            'Star-Lord': star_lord,
            'Sub-Lord': sub_lord,
            'Sub-Sub-Lord': ss_lord,
            'Degree': d,
            'Min': m,
            'Second': s,
            'KS-N': ks_n,
            'CIL-N': cil_n,
            'KS-D': ks_d,
            'paada': paada_val
        }

    # ==============================
    # 3. 主循环
    # ==============================
    rows = []

    # 全局计数器，用于生成 CIL-N
    # 逻辑：每次进入新的 Sub-Sub 周期 +1；如果周期内发生“换座”，再额外 +1
    cil_n_counter = 0

    global_ks_n_counter = 0

    TOTAL_YEARS = Fraction(120, 1)

    for star_idx in range(27):
        star_name = STARS[star_idx]
        star_lord = LORDS[star_idx % 9]
        star_start_abs = Fraction(star_idx, 1) * STAR_SEC

        sub_seq = get_lord_sequence(star_lord)
        current_sub_offset = Fraction(0, 1)

        for sub_lord in sub_seq:
            # A. Sub 范围计算
            sub_len = (Fraction(YEARS[sub_lord], 1) / TOTAL_YEARS) * STAR_SEC
            sub_abs_start = star_start_abs + current_sub_offset
            sub_abs_end = sub_abs_start + sub_len

            # B. KS 预判
            sub_start_sign_idx = int(sub_abs_start // SIGN_SEC)
            check_end = sub_abs_end
            if check_end % SIGN_SEC == 0:
                check_end -= 1
            sub_end_sign_idx = int(check_end // SIGN_SEC)
            is_sub_split = (sub_start_sign_idx != sub_end_sign_idx)

            ks_n_part1 = global_ks_n_counter + 1
            ks_d_part1 = float(sub_abs_start) / 3600.0
            ks_n_part2 = None
            ks_d_part2 = None

            if is_sub_split:
                global_ks_n_counter += 2
                ks_n_part2 = ks_n_part1 + 1
                abs_boundary_sec = (sub_start_sign_idx + 1) * SIGN_SEC
                ks_d_part2 = float(abs_boundary_sec) / 3600.0
            else:
                global_ks_n_counter += 1

            def get_labels(current_sign_idx):
                if current_sign_idx == sub_start_sign_idx:
                    return ks_n_part1, ks_d_part1
                else:
                    return ks_n_part2, ks_d_part2

            # C. Sub-Sub 循环
            ss_seq = get_lord_sequence(sub_lord)
            current_ss_offset = Fraction(0, 1)

            for ss_lord in ss_seq:
                ss_len = (Fraction(YEARS[ss_lord], 1) / TOTAL_YEARS) * sub_len
                abs_start = sub_abs_start + current_ss_offset
                abs_end = abs_start + ss_len

                if abs_end > ZODIAC_SEC: abs_end = ZODIAC_SEC

                # 初始化当前 SS 周期的基础 CIL-N
                cil_n_counter += 1
                base_cil_n = cil_n_counter

                # D. 高级切分逻辑
                # 1. 找出所有切分点 (Sign + Paada)
                first_k = int(abs_start // PAADA_SEC) + 1
                next_boundary = first_k * PAADA_SEC

                boundaries = []
                curr_b = next_boundary
                while curr_b < abs_end:
                    boundaries.append(curr_b)
                    curr_b += PAADA_SEC

                # 2. 生成片段
                segments = []
                curr_s = abs_start
                for b in boundaries:
                    segments.append((curr_s, b))
                    curr_s = b
                segments.append((curr_s, abs_end))

                # 3. 遍历片段生成行，处理 CIL-N 继承规则
                prev_sign_idx = -1

                for i, (seg_s, seg_e) in enumerate(segments):
                    if seg_s >= seg_e: continue

                    current_sign_idx = int(seg_s // SIGN_SEC)

                    if i == 0:
                        # 第一段：使用分配的基础 ID
                        current_row_cil = base_cil_n
                    else:
                        # 后续段：检查是否换座
                        if current_sign_idx != prev_sign_idx:
                            # 换座 -> CIL-N 递增 (占星学派逻辑：跨座即新事件)
                            cil_n_counter += 1
                            base_cil_n = cil_n_counter
                            current_row_cil = base_cil_n
                        else:
                            # 仅 Paada 变化 -> CIL-N 继承 (同一星座内的细分)
                            current_row_cil = base_cil_n

                    prev_sign_idx = current_sign_idx

                    # 计算 Paada
                    rel_star_pos = seg_s - star_start_abs
                    paada_val = int(rel_star_pos // PAADA_SEC) + 1

                    # 严谨处理：由于浮点微差或边界，确保不超出 1-4 范围
                    if paada_val > 4: paada_val = 4
                    if paada_val < 1: paada_val = 1

                    k_n, k_d = get_labels(current_sign_idx)

                    rows.append(format_row(
                        seg_s, seg_e, current_sign_idx,
                        star_name, star_lord, sub_lord, ss_lord,
                        current_row_cil, k_n, k_d, paada_val
                    ))

                current_ss_offset += ss_len

            current_sub_offset += sub_len

    return pd.DataFrame(rows)

if __name__ == "__main__":
    df = generate_kp_table_paada_strict_final()
    filename = 'KP_Table_Strict_Paada_Final.csv'
    df.to_csv(filename, index=False)
    print(f"File generated: {filename}")
    print(f"Total Rows: {len(df)}")
    print(f"Max CIL-N: {df['CIL-N'].max()}")

File generated: KP_Table_Strict_Paada_Final.csv
Total Rows: 2256
Max CIL-N: 2193
